In [ ]:
import linpde_gp
import numpy as np
import probnum as pn
from gp_fvm.utils.figure_manager import FigureManager
from gp_fvm.plot.dim1 import plot_function, compare_to_solution_gp, plot_gp

pn.config.default_solver_linpde_gp = linpde_gp.solvers.CholeskySolver(dense=True)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

data_path = os.environ.get("DATA_DIR")

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
spatial_domain = linpde_gp.domains.asdomain([0.0, 1.0])
temporal_domain = linpde_gp.domains.asdomain([0., 2.0])
beta = 0.4

domain = linpde_gp.domains.Box([temporal_domain, spatial_domain])

In [ ]:
from linpde_gp.linfuncops.diffops import LinearDifferentialOperator, PartialDerivativeCoefficients, MultiIndex
D = LinearDifferentialOperator(
    coefficients=PartialDerivativeCoefficients(
        {
            (): {MultiIndex((1, 0)): 1.0, MultiIndex((0, 1)): beta},
        },
        (2,),
        (),
    ),
    input_shapes=((2,), ()),
)

In [ ]:
from advection_utils import read_data, get_problem, get_ts, get_xs

In [ ]:
from linpde_gp.linfunctls import (  # pylint: disable=import-outside-toplevel
    _EvaluationFunctional,
)

def periodic_boundary_functional(X_left, X_right):
    L_left = _EvaluationFunctional(
            input_domain_shape=(2,),
            input_codomain_shape=(),
            X=X_left,
        )
    L_right = _EvaluationFunctional(
            input_domain_shape=(2,),
            input_codomain_shape=(),
            X=X_right,
        )
    return L_left - L_right

In [ ]:
lengthscale_t = 0.3
lengthscale_x = 0.2
output_scale = 1.0

ic_stride = 10 # 1/10th of the original data
N_bc = 50

def get_prior(l_t, l_x, output_scale):
    return pn.randprocs.GaussianProcess(
        mean=linpde_gp.functions.Zero(input_shape=(2,)),
        cov=output_scale**2
        * linpde_gp.randprocs.covfuncs.TensorProduct(
            linpde_gp.randprocs.covfuncs.Matern((), nu=1.5, lengthscales=l_t),
            linpde_gp.randprocs.covfuncs.Matern((), nu=1.5, lengthscales=l_x),
        ),
    )

u_prior = get_prior(lengthscale_t, lengthscale_x, output_scale)

In [ ]:
hf = read_data(f"{data_path}/1D_Advection_Sols_beta0.4.hdf5")
problem_idx = 42
problem = get_problem(hf, problem_idx)
ts = get_ts(hf)
xs = get_xs(hf)
problem[0][::12].shape, xs[::12].shape

In [ ]:
from linpde_gp.randprocs.covfuncs import TensorProductGrid
X_ic = TensorProductGrid([0.0], xs[::ic_stride].astype(np.float64))
Y_ic = problem[0][::ic_stride].astype(np.float64)
Y_ic = Y_ic.reshape(X_ic.shape[:-1])

In [ ]:
plt.plot(X_ic[0, :, 1], Y_ic.reshape(-1))

In [ ]:
u_ic = u_prior.condition_on_observations(Y_ic, X_ic)
plot_gp(u_ic, domain, num_t=3)

In [ ]:
N_bc = 50
X_left = domain.uniform_grid((N_bc, 1))
X_right = domain.uniform_grid((N_bc, 1), inset=(0, domain.bounds[1][1]))

L_boundary = periodic_boundary_functional(X_left, X_right)
Y_boundary = np.zeros(L_boundary.output_shape)

u_ic_bc = u_ic.condition_on_observations(Y=Y_boundary, L=L_boundary)
plot_gp(u_ic_bc, domain, num_t=3)

In [ ]:
from gp_fvm.finite_volumes import get_grid_from_resolution
from linpde_gp.linfunctls import FiniteVolumeFunctional

domains = get_grid_from_resolution(domain, [80, 80])
fv = FiniteVolumeFunctional(domains, D)
u_all = u_ic_bc.condition_on_observations(L=fv, Y=np.zeros(domains.shape))
plot_gp(u_all, domain, num_t=3)

In [ ]:
from gp_fvm.plot.dim1 import plot_at_time
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def animate_gp_sol(gp):
    fig, ax = plt.subplots()

    fps = 30
    X_eval = domain.uniform_grid((int(fps * domain.bounds[0][1]), 40))
    print("Mean computation...")
    means = gp.mean(X_eval)
    print("Stds computation...")
    stds = np.sqrt(gp.cov.linop(X_eval).diagonal())
    stds = stds.reshape(means.shape)
    print("Done")

    def update(frame):
        ax.clear()
        ax.plot(X_eval[frame, :, 1], means[frame])
        ax.fill_between(
            X_eval[frame, :, 1],
            means[frame] - 1.96 * stds[frame],
            means[frame] + 1.96 * stds[frame],
            alpha=0.2,
            color='cornflowerblue',
        )
        # ax.set_title(f't = {X_eval[frame, :, 0]:.2f} sec')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('u(t, x)')

    interval = 1000/fps
    return FuncAnimation(fig, update, frames=X_eval.shape[0], interval=interval)


In [ ]:
HTML(animate_gp_sol(u_all).to_jshtml())

In [ ]:
def gp_sol_errors(gp, sol, ts, xs, method="Linf"):
    """
        Returns a tuple of (Linf, MSE) errors
    """
    assert sol.shape == (ts.size, xs.size)
    X_eval = TensorProductGrid(ts, xs)
    gp_preds = gp.mean(X_eval)
    if method == "Linf":
        return np.abs(gp_preds - sol).max()
    elif method == "MSE":
        return ((gp_preds - sol)**2).mean()
    else:
        raise Exception("Unsupported method: " + method)

In [ ]:
gp_sol_errors(u_all, problem, ts.astype(np.float64), xs.astype(np.float64), method="Linf")